In [1]:
### KEY ITEMS DEFINED
#libraries
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import math

#tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pearsonr
#models
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
#defining key models
lr = LinearRegression()
knn = KNeighborsRegressor(n_neighbors=7)
dt = DecisionTreeRegressor(max_depth=10,random_state=27)
rf = RandomForestRegressor(n_estimators=20, random_state=1)
#PCC df and lists
pcc_df = pd.DataFrame(columns=["PCC"])
mae_df = pd.DataFrame(columns=["MAE"])
rmse_df = pd.DataFrame(columns=["RMSE"])
lr_avg_pcc = []
knn_avg_pcc = []
dt_avg_pcc = []
rf_avg_pcc = []
lr_avg_mae = []
knn_avg_mae= []
dt_avg_mae = []
rf_avg_mae = []
lr_avg_rmse = []
knn_avg_rmse= []
dt_avg_rmse = []
rf_avg_rmse = []

#files
files = ['BioEngineeringPV', 'BSB_BuildingPV', 'BSB_LibraryPV', 'CSC_BuildingPV', 'CUP_PV', 'EBU2_A_PV', 'EBU2_B_PV', 'ElectricShopPV', 'GarageFleetsPV', 'GilmanParkingPV', 'HopkinsParkingPV', 'KeelingA_PV', 'KeelingB_PV', 'KyoceraSkylinePV', 'LeichtagPV', 'MayerHallPV', 'MESOM_PV', 'OslerParkingPV', 'PowellPV', 'PriceCenterA_PV', 'PriceCenterB_PV', 'SDSC_PV', 'SME_SolarPV', 'StephenBirchPV']

#workflow function
def workflow(x,y):
    #defining global variables
    global y_test
    global y_pred
    #data prep
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=False)
    x_train = x_train[:130843]
    y_train = y_train[:130843]
    #create normalizer
    min_max_scaler = MinMaxScaler()
    #normalize data
    x_train_norm = min_max_scaler.fit_transform(x_train)
    x_test_norm = min_max_scaler.fit_transform(x_test)

    #LR model
    #create model
    model = lr
    #print model name
    print("Model:", model)
    #fit model to train data
    model.fit(x_train_norm, y_train)
    #predict
    y_pred = model.predict(x_test_norm)
    #reshape
    y_pred = np.array(y_pred).reshape(-1)
    y_test = np.array(y_test).reshape(-1)
    #evaluate
    corr, _ = pearsonr(y_pred, y_test)
    pcc = ('%.4f' % corr)
    lr_avg_pcc.append(pcc)
    lr_avg_mae.append(mean_absolute_error(y_test,y_pred))
    lr_avg_rmse.append(math.sqrt(mean_squared_error(y_test,y_pred)))

    #KNN model
    #create model
    model = knn
    #print model name
    print("Model:", model)
    #fit model to train data
    model.fit(x_train_norm, y_train)
    #predict
    y_pred = model.predict(x_test_norm)
    #reshape
    y_pred = np.array(y_pred).reshape(-1)
    y_test = np.array(y_test).reshape(-1)
    #evaluate
    corr, _ = pearsonr(y_pred, y_test)
    pcc = ('%.4f' % corr)
    knn_avg_pcc.append(pcc)
    knn_avg_mae.append(mean_absolute_error(y_test,y_pred))
    knn_avg_rmse.append(math.sqrt(mean_squared_error(y_test,y_pred)))

    #DT model
    #create model
    model = dt
    #print model name
    print("Model:", model)
    #fit model to train data
    model.fit(x_train_norm, y_train)
    #predict
    y_pred = model.predict(x_test_norm)
    #reshape
    y_pred = np.array(y_pred).reshape(-1)
    y_test = np.array(y_test).reshape(-1)
    #evaluate
    corr, _ = pearsonr(y_pred, y_test)
    pcc = ('%.4f' % corr)
    dt_avg_pcc.append(pcc)
    dt_avg_mae.append(mean_absolute_error(y_test,y_pred))
    dt_avg_rmse.append(math.sqrt(mean_squared_error(y_test,y_pred)))

    #RF model
    #create model
    model = rf
    #print model name
    print("Model:", model)
    #fit model to train data
    model.fit(x_train_norm, y_train)
    #predict
    y_pred = model.predict(x_test_norm)
    #reshape
    y_pred = np.array(y_pred).reshape(-1)
    y_test = np.array(y_test).reshape(-1)
    #evaluate
    corr, _ = pearsonr(y_pred, y_test)
    pcc = ('%.4f' % corr)
    rf_avg_pcc.append(pcc)
    rf_avg_mae.append(mean_absolute_error(y_test,y_pred))
    rf_avg_rmse.append(math.sqrt(mean_squared_error(y_test,y_pred)))

In [2]:
present_future_list = np.linspace(1,96,96, dtype=int)

In [3]:
for file in files:
    #loading data
    df = pd.read_csv('/Users/farihahahmed/Desktop/PVGenerator/%s.csv' % file) 
    df["DateTime"] = pd.to_datetime(df["DateTime"])
    df.sort_values(by="DateTime", inplace=True)
    df.reset_index(inplace=True,drop=True)
    #delete unnecessary datetime column
    df.drop("DateTime", inplace=True, axis=1)


    #create present and future columns (duplicates)
    for present in present_future_list:
        df['Present-%s' % present] = df['RealPower']
        df['Present-%s' % present] = df['Present-%s' % present].shift(-abs(present-1))
    for future in present_future_list:
        df['Future-%s' % future] = df['RealPower']
        df['Future-%s' % future] = df['Future-%s' % future].shift(-abs(future-1))
    #drop RealPower column
    df = df.drop(['RealPower'], axis=1)
    #delete last 191 rows (192 inputs)
    df = df.head(df.shape[0] -191)
    #interpolate if there are any missing values
    df = df.interpolate(method ='linear', limit_direction ='forward')
    #create input and output
    x = df[df.columns[:96]]
    y = df[df.columns[96:]]
    #workflow
    workflow(x,y)    

/var/folders/vy/kn2fzcwd0h52b0p6lxcdn5_80000gn/T/ipykernel_40822/2363790593.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Future-%s' % future] = df['RealPower']
/var/folders/vy/kn2fzcwd0h52b0p6lxcdn5_80000gn/T/ipykernel_40822/2363790593.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Future-%s' % future] = df['RealPower']
/var/folders/vy/kn2fzcwd0h52b0p6lxcdn5_80000gn/T/ipykernel_40822/2363790593.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` m

Model: LinearRegression()
Model: KNeighborsRegressor(n_neighbors=7)
Model: DecisionTreeRegressor(max_depth=10, random_state=27)
Model: RandomForestRegressor(n_estimators=20, random_state=1)


/var/folders/vy/kn2fzcwd0h52b0p6lxcdn5_80000gn/T/ipykernel_40822/2363790593.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Future-%s' % future] = df['RealPower']
/var/folders/vy/kn2fzcwd0h52b0p6lxcdn5_80000gn/T/ipykernel_40822/2363790593.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Future-%s' % future] = df['RealPower']
/var/folders/vy/kn2fzcwd0h52b0p6lxcdn5_80000gn/T/ipykernel_40822/2363790593.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` m

Model: LinearRegression()
Model: KNeighborsRegressor(n_neighbors=7)
Model: DecisionTreeRegressor(max_depth=10, random_state=27)
Model: RandomForestRegressor(n_estimators=20, random_state=1)


/var/folders/vy/kn2fzcwd0h52b0p6lxcdn5_80000gn/T/ipykernel_40822/2363790593.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Future-%s' % future] = df['RealPower']
/var/folders/vy/kn2fzcwd0h52b0p6lxcdn5_80000gn/T/ipykernel_40822/2363790593.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Future-%s' % future] = df['RealPower']
/var/folders/vy/kn2fzcwd0h52b0p6lxcdn5_80000gn/T/ipykernel_40822/2363790593.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` m

...

Model: LinearRegression()
Model: KNeighborsRegressor(n_neighbors=7)
Model: DecisionTreeRegressor(max_depth=10, random_state=27)
Model: RandomForestRegressor(n_estimators=20, random_state=1)


/var/folders/vy/kn2fzcwd0h52b0p6lxcdn5_80000gn/T/ipykernel_40822/2363790593.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Future-%s' % future] = df['RealPower']
/var/folders/vy/kn2fzcwd0h52b0p6lxcdn5_80000gn/T/ipykernel_40822/2363790593.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Future-%s' % future] = df['RealPower']
/var/folders/vy/kn2fzcwd0h52b0p6lxcdn5_80000gn/T/ipykernel_40822/2363790593.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` m

Model: LinearRegression()
Model: KNeighborsRegressor(n_neighbors=7)
Model: DecisionTreeRegressor(max_depth=10, random_state=27)
Model: RandomForestRegressor(n_estimators=20, random_state=1)


/var/folders/vy/kn2fzcwd0h52b0p6lxcdn5_80000gn/T/ipykernel_40822/2363790593.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Future-%s' % future] = df['RealPower']
/var/folders/vy/kn2fzcwd0h52b0p6lxcdn5_80000gn/T/ipykernel_40822/2363790593.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Future-%s' % future] = df['RealPower']
/var/folders/vy/kn2fzcwd0h52b0p6lxcdn5_80000gn/T/ipykernel_40822/2363790593.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` m

Model: LinearRegression()
Model: KNeighborsRegressor(n_neighbors=7)
Model: DecisionTreeRegressor(max_depth=10, random_state=27)
Model: RandomForestRegressor(n_estimators=20, random_state=1)


In [4]:
pcc_df.loc['LR']=np.mean(np.array(lr_avg_pcc).astype(float))
pcc_df.loc['KNN']=np.mean(np.array(knn_avg_pcc).astype(float))
pcc_df.loc['DT']=np.mean(np.array(dt_avg_pcc).astype(float))
pcc_df.loc['RF']=np.mean(np.array(rf_avg_pcc).astype(float))
print(pcc_df)

          PCC
LR   0.999942
KNN  0.989433
DT   0.943129
RF   0.968763


In [5]:
mae_df.loc['LR']=np.mean(np.array(lr_avg_mae).astype(float))
mae_df.loc['KNN']=np.mean(np.array(knn_avg_mae).astype(float))
mae_df.loc['DT']=np.mean(np.array(dt_avg_mae).astype(float))
mae_df.loc['RF']=np.mean(np.array(rf_avg_mae).astype(float))
print(mae_df)

          MAE
LR   3.348025
KNN  3.043927
DT   4.146146
RF   3.343690


In [6]:
rmse_df.loc['LR']=np.mean(np.array(lr_avg_rmse).astype(float))
rmse_df.loc['KNN']=np.mean(np.array(knn_avg_rmse).astype(float))
rmse_df.loc['DT']=np.mean(np.array(dt_avg_rmse).astype(float))
rmse_df.loc['RF']=np.mean(np.array(rf_avg_rmse).astype(float))
print(rmse_df)

         RMSE
LR   5.447444
KNN  5.428821
DT   7.424124
RF   5.633322
